## Real World Prompting

### Real-World Prompting Tips

* Let LLM think
* Be clear and direct
* Use Prompt templates
* Give the model a role
* Structure prompts with XML
* Chain complex prompts
* Use examples

In [4]:
import os
from openai import OpenAI

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")

client = OpenAI(api_key=dashscope_api_key, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
QWEN_MODEL = "qwen-plus"
QWQ_MODEL = "qwq-plus"
MULTIMODAL = "qwen-vl-max"

#client = OpenAI(api_key = deepseek_api_key, base_url = "https://api.deepseek.com")
# R1_MODEL = "deepseek-reasoner"
# V3_MODEL = "deepseek-chat"

### A Consumer Prompt

In [5]:
consumer_prompt = "help me brainstorm ideas for a talk on AI and education"

### An Enterprise-Grade Prompt

In [6]:
enterprise_prompt = """
请分析以下客户服务通话记录，并生成一份交互摘要的 JSON 格式总结：

<transcript>
[在此插入通话记录]
</transcript>

说明：
<instructions>
1. 仔细阅读通话记录。
2. 分析通话内容，聚焦于主要问题、解决方案以及是否需要后续跟进。
3. 根据指定的结构，生成一个 JSON 对象，概括此次交流的关键要点。

重要规范：
- 保密性：请省略所有客户的个人信息，如姓名、电话号码和邮箱地址。
- 字符限制：每个文本字段最多不超过 100 个字符。
- 专业语气：请保持总结内容的专业表达。

输出格式：
请生成如下结构的 JSON 对象：
<json>
{
  "summary": {
    "customerIssue": "简要描述客户来电的主要问题",
    "resolution": "说明问题如何被处理或解决（如有）",
    "followUpRequired": true/false,
    "followUpDetails": "如需后续操作，请描述相关事项；如无需，填写 null"
  },
  "status": "COMPLETE",
  "ambiguities": ["列出对话中存在的不清楚或模糊的点；如无，请填写空数组"]
}
</json>

信息不足判定标准：
如满足以下任一条件：
a) 通话总往返不满 5 轮  
b) 客户的问题不明确  
c) 通话内容模糊不清、不完整或受到语言障碍影响  
则仅返回以下 JSON 内容：
{
  "status": "INSUFFICIENT_DATA"
}

示例：
<examples>
1. 完整通话案例：
<transcript>
客服：感谢致电 Acme 智能家居支持中心，我是 Alex，请问有什么可以帮您的？
客户：你好 Alex，我的 Acme SmartTherm 无法维持我设置的温度。我设定了 72 度，但屋里感觉热多了。
客服：很抱歉听到这个问题。我们来一起排查一下。SmartTherm 现在连着 Wi-Fi 吗？
客户：是的，显示屏上有 Wi-Fi 的图标。
客服：很好。我们来重新校准设备。请按住菜单键 5 秒钟。
客户：好了，出现了一个新菜单。
客服：很好。请进入“校准”选项，按确认键。然后根据室温计调整温度。
客户：好了，我调到了 79 度以匹配室温计。
客服：太好了。请按确认键确认。重新校准需要几分钟。请一个小时后再查看是否恢复正常。
客户：好的，谢谢你的帮助，Alex。
客服：不客气！还有其他需要帮忙的吗？
客户：没有了，谢谢。
客服：感谢您选择 Acme 智能家居，祝您有美好的一天！
</transcript>

<thinking>
主要问题：SmartTherm 无法维持设定的温度  
解决方案：指导客户完成了设备的重新校准  
后续跟进：不需要，但建议客户稍后检查是否恢复正常  
模糊点：无
</thinking>

<json>
{
  "summary": {
    "customerIssue": "SmartTherm 无法维持设定温度，高于设定的 72 度",
    "resolution": "指导客户完成了 SmartTherm 的重新校准操作",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. 需要后续跟进的通话：
<transcript>
客服：您好，这里是 Acme 智能家居支持，我是 Jamie，请问有什么可以帮您？
客户：你好，我刚安装了新的 SmartCam，但它连接不上 Wi-Fi。
客服：我来帮您处理。您使用的是 Acme 的智能家居 App 吗？
客户：是的，我手机上有这个 App。
客服：很好，请确保您的手机连的是 2.4GHz 的 Wi-Fi 网络，而不是 5GHz。
客户：哦，我现在是连着 5GHz。我要切换吗？
客服：是的，请切换到 2.4GHz。SmartCam 只支持 2.4GHz。
客户：好了，切换好了。接下来怎么做？
客服：打开 App，选择“添加设备”，选择“SmartCam”，按照提示操作。
客户：它现在要我输入密码。
客服：请您输入 Wi-Fi 密码，应该就可以连接了。
客户：还是不行，我一直收到错误提示。
客服：明白了。我将把这个问题转交给我们的技术团队，他们会在 24 小时内联系您。
客户：好的，谢谢你的帮助。
客服：不客气，还有其他需要协助的吗？
客户：暂时没有了，谢谢。
客服：感谢您选择 Acme 智能家居，祝您愉快！
</transcript>

<thinking>
主要问题：客户无法将新的 SmartCam 连接到 Wi-Fi  
解决方案：初步排查未能解决，已将问题升级至技术团队处理  
后续跟进：需要，技术团队将在 24 小时内联系客户  
模糊点：客户收到的具体错误信息未说明
</thinking>

<json>
{
  "summary": {
    "customerIssue": "无法将新 SmartCam 连接到 Wi-Fi",
    "resolution": "初步排查未解决，已上报技术团队",
    "followUpRequired": true,
    "followUpDetails": "技术团队将在 24 小时内联系客户提供进一步支持"
  },
  "status": "COMPLETE",
  "ambiguities": ["客户遇到的具体错误信息未说明"]
}
</json>

3. 信息不足的通话：
<transcript>
客服：您好，这里是 Acme 智能家居支持，我是 Sam，请问有什么可以帮您？
客户：我的智能门锁不工作了。
客服：很抱歉，请您详细说说是什么问题？
客户：它就是不能用了。我也说不清楚。
客服：那您记得是从什么时候开始不工作的？您使用的是哪个型号的门锁？
客户：我不记得了。我现在得走了，晚点再打吧。
客服：好的，我们全天候为您服务，祝您愉快。
</transcript>

<thinking>
该通话往返不足 5 轮，客户的问题表述不清，也没有回应客服的进一步问题。缺乏足够信息，无法生成完整摘要。
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
</instructions>

在生成 JSON 前，请先使用 <thinking> 标签对通话进行分析，明确主要问题、解决方式、是否需要后续跟进及存在的模糊点。  
然后再使用 <json> 标签输出 JSON 结果。
"""

## Our Goal

### Sample Customer Reviews

I recently purchased the XYZ Smartphone, and I have to say, it's been a mixed experience. On the positive side, the camera quality is absolutely stunning. The photos I've taken are crisp, clear, and the colors are vibrant. The battery life is also impressive, lasting me a full day even with heavy use.

However, there are a few issues that have been frustrating. Firstly, the phone tends to overheat when I'm using multiple apps or playing games for extended periods. It becomes uncomfortably warm to hold. Secondly, the facial recognition feature is unreliable. It often fails to recognize me in low light conditions, forcing me to use the PIN instead.

The user interface is sleek and intuitive, which I appreciate. But I've noticed occasional lag when switching between apps, which is disappointing for a phone at this price point. Customer support has been helpful in addressing some of my concerns, but they haven't been able to resolve the overheating issue.

Overall, while I'm impressed with certain aspects of the phone, the negatives are significant enough that I'm not sure I can wholeheartedly recommend it. It's a decent phone, but for the price, I expected fewer issues.

### Desired Output

```JSON
{
  "sentiment_score": "Negative",
  "sentiment_analysis": "While the review acknowledges positive aspects (camera quality, battery life, and user interface), the overall sentiment is negative due to multiple unresolved technical issues, performance problems, and price-to-value concerns. The reviewer explicitly states reluctance to recommend the product and concludes with disappointment regarding the value proposition.",
  "complaints": [
    "Phone overheating during multiple apps usage and gaming",
    "Unreliable facial recognition system, especially in low light",
    "Performance lag when switching between apps",
    "Poor value for money considering the price point"
  ]
}
```

## Writing Our Prompt

### Set The Role

In [7]:
setting_the_role = """
You are an AI assistant specialized in analyzing customer reviews.
Your task is to determine the overall sentimnet of a given review
and extract any specific compliants mentioned.
Please follow these instructions carefully:
"""

In [13]:
setting_the_role_zh = """
你是一名专门分析客户评论的 AI 助手。  
你的任务是判断给定评论的整体情感倾向，  
并提取出其中提到的具体投诉内容。  
请仔细遵循以下说明：
"""

### Make It A Prompt Template

In [8]:
instruction_pt1 = """
1. Review the following customer feedback:

<customer_review>
{{CUSTOMER_REVIEW}}
</customer_review>
"""

In [14]:
instruction_pt1_zh = """
1. 阅读以下客户反馈：

<customer_review>
{{CUSTOMER_REVIEW}}
</customer_review>
"""

### Let LLM Think

In [9]:
instruction_pt2 = """
2. Analyze the review using the following steps. 
Show your work in <review_breakdown> tags:

a) Key Phrase Extraction:
   - Extract and quote key phrases that indicate sentiment 
   (positive, negative, or neutral).
   - Extract and quote key phrases that suggest complaints or issues.

b) Sentiment Analysis:
   - Consider arguments for positive, negative, 
   and neutral sentiment based on the extracted phrases.
   - Determine the overall sentiment (positive, negative, or neutral) 
   based on your analysis.
   - Explain your reasoning for the sentiment classification.

c) Complaint Extraction:
   - List each specific issue or problem mentioned in the review.
   - For each complaint, provide the relevant quote from the review.
   - Count the total number of complaints found.

It's OK for this section to be quite long as you 
thoroughly break down the review.
"""

In [15]:
instruction_pt2_zh = """
2. 使用以下步骤分析评论内容。  
请将分析过程置于 <review_breakdown> 标签中展示：

a) 关键词提取：
   - 提取并引用表明情感倾向（正面、负面或中立）的关键词或短语；
   - 提取并引用暗示投诉或问题的关键词或短语。

b) 情感分析：
   - 根据提取的关键词，分别考虑支持正面、负面和中立情感的论点；
   - 根据分析结果，确定整体情感倾向（正面、负面或中立）；
   - 解释你选择该情感分类的理由。

c) 投诉提取：
   - 列出评论中提到的每一个具体问题或投诉；
   - 对于每项投诉，提供评论中的对应引用句；
   - 统计发现的投诉总数。

在这个部分可以写得比较详细，以全面深入地分析评论内容。
"""

### Output Instructions

In [10]:
instruction_pt3 = """
3. Based on your analysis, 
generate a JSON output with the following structure:

<json>
{
  "sentiment_score": "Positive|Negative|Neutral",
  "sentiment_analysis": "Explanation of sentiment classification",
  "complaints": [
    "Complaint 1",
    "Complaint 2",
    "..."
  ]
}
</json>

If no complaints are found, 
use an empty array for the "complaints" field.

Remember:
- Base your analysis solely on the content of the provided review.
- Do not make assumptions or include information 
not present in the review.
- Be objective and focus on the customer's expressed 
opinions and experiences.
- Ensure your JSON output is properly formatted 
and contains all required fields.
"""

In [16]:
instruction_pt3_zh = """
3. 根据你的分析，生成以下结构的 JSON 输出：

<json>
{
  "sentiment_score": "Positive|Negative|Neutral",
  "sentiment_analysis": "情感分类的解释说明",
  "complaints": [
    "投诉1",
    "投诉2",
    "..."
  ]
}
</json>

如果未发现任何投诉，请将 "complaints" 字段设置为空数组。

请记住：
- 分析必须仅基于提供的评论内容；
- 不要做任何假设，也不要加入评论中未提到的信息；
- 保持客观，专注于客户表达的意见与体验；
- 确保你的 JSON 输出格式正确，并包含所有必要字段。
"""

### Assembling The Prompt

In [11]:
final_prompt = f"""
{setting_the_role}
{instruction_pt1}
{instruction_pt2}
{instruction_pt3}
"""

In [12]:
print(final_prompt)



You are an AI assistant specialized in analyzing customer reviews.
Your task is to determine the overall sentimnet of a given review
and extract any specific compliants mentioned.
Please follow these instructions carefully:


1. Review the following customer feedback:

<customer_review>
{{CUSTOMER_REVIEW}}
</customer_review>


2. Analyze the review using the following steps. 
Show your work in <review_breakdown> tags:

a) Key Phrase Extraction:
   - Extract and quote key phrases that indicate sentiment 
   (positive, negative, or neutral).
   - Extract and quote key phrases that suggest complaints or issues.

b) Sentiment Analysis:
   - Consider arguments for positive, negative, 
   and neutral sentiment based on the extracted phrases.
   - Determine the overall sentiment (positive, negative, or neutral) 
   based on your analysis.
   - Explain your reasoning for the sentiment classification.

c) Complaint Extraction:
   - List each specific issue or problem mentioned in the review.


In [17]:
final_prompt_zh = f"""
{setting_the_role_zh}
{instruction_pt1_zh}
{instruction_pt2_zh}
{instruction_pt3_zh}
"""

In [18]:
print(final_prompt_zh)



你是一名专门分析客户评论的 AI 助手。  
你的任务是判断给定评论的整体情感倾向，  
并提取出其中提到的具体投诉内容。  
请仔细遵循以下说明：


1. 阅读以下客户反馈：

<customer_review>
{{CUSTOMER_REVIEW}}
</customer_review>


2. 使用以下步骤分析评论内容。  
请将分析过程置于 <review_breakdown> 标签中展示：

a) 关键词提取：
   - 提取并引用表明情感倾向（正面、负面或中立）的关键词或短语；
   - 提取并引用暗示投诉或问题的关键词或短语。

b) 情感分析：
   - 根据提取的关键词，分别考虑支持正面、负面和中立情感的论点；
   - 根据分析结果，确定整体情感倾向（正面、负面或中立）；
   - 解释你选择该情感分类的理由。

c) 投诉提取：
   - 列出评论中提到的每一个具体问题或投诉；
   - 对于每项投诉，提供评论中的对应引用句；
   - 统计发现的投诉总数。

在这个部分可以写得比较详细，以全面深入地分析评论内容。


3. 根据你的分析，生成以下结构的 JSON 输出：

<json>
{
  "sentiment_score": "Positive|Negative|Neutral",
  "sentiment_analysis": "情感分类的解释说明",
  "complaints": [
    "投诉1",
    "投诉2",
    "..."
  ]
}
</json>

如果未发现任何投诉，请将 "complaints" 字段设置为空数组。

请记住：
- 分析必须仅基于提供的评论内容；
- 不要做任何假设，也不要加入评论中未提到的信息；
- 保持客观，专注于客户表达的意见与体验；
- 确保你的 JSON 输出格式正确，并包含所有必要字段。




## Using The Prompt

In [19]:
import re

def get_review_sentiment(review):
    # Insert the context into prompt
    prompt = final_prompt.replace("{{CUSTOMER_REVIEW}}", review)
    # Send a request to Client
    response = client.chat.completions.create(
        model = QWEN_MODEL,
        max_tokens = 2000,
        messages = [
            {"role": "user", 
            "content": prompt}
        ],
    )

    output = response.choices[0].message.content
    print("ENTIRE MODEL OUTPUT: ")
    print(output)

    # 使用正则表达式搜索文本中 <json> 和 </json> 标签之间的内容，re.DOTALL 让 . 匹配换行符
    sentiment = re.search(r'<json>(.*?)</json>', output, re.DOTALL)

    # 如果找到了匹配的内容（即找到了 <json> 标签对）
    if sentiment:
        print("FINAL JSON OUTPUT: ")
        print(sentiment.group(1).strip()) # 输出提取出的 JSON 内容，去除前后空白字符
    else:
        print("No sentiment analysis in the response.")

In [20]:
review1 = """
I am in love with my Acme phone.  It's incredible.
It's a little expensive, but so worth it imo.
If you can afford it, it's worth it! 
I love the colors too!
"""

In [21]:
get_review_sentiment(review1)

ENTIRE MODEL OUTPUT: 
<review_breakdown>

a) Key Phrase Extraction:

- Positive Sentiment Phrases:
  - "I am in love with my Acme phone."
  - "It's incredible."
  - "so worth it imo."
  - "If you can afford it, it's worth it!"
  - "I love the colors too!"

- Neutral/Issue Suggestion Phrase:
  - "It's a little expensive,"

b) Sentiment Analysis:

- Arguments for Positive Sentiment:
  The review contains multiple strong positive phrases such as "I am in love with my Acme phone," "It's incredible," and "so worth it imo." The customer explicitly states their satisfaction with the product, particularly highlighting its value despite the cost. Additionally, they express enthusiasm about the colors of the phone.

- Arguments for Negative Sentiment:
  There is one phrase that could be seen as slightly negative: "It's a little expensive." However, this is immediately followed by the customer justifying the cost ("but so worth it imo"), which mitigates any potential negativity.

- Arguments for 

In [22]:
review2 = """
I recently bought the ABC Smartwatch, and I have to say, it's been a 
bit of a rollercoaster. Let's start with the good stuff - 
the design is absolutely gorgeous. It's sleek, lightweight, 
and looks fantastic on my wrist. The display is crisp and bright, 
even in direct sunlight. I also love how customizable the watch 
faces are, allowing me to switch up the look whenever I want.
However, there are some significant downsides that I can't ignore. 
The battery life is terrible - I'm lucky if I get through a full 
day without needing to charge it. This is especially frustrating
when I'm traveling or out for long periods. The fitness tracking 
features, which were a big selling point for me, are hit or miss. 
It often doesn't accurately count my steps or calculate calories burned 
during workouts. Another issue I've encountered is with the touch 
sensitivity. Sometimes it's overly sensitive, registering accidental 
touches, while other times I have to tap multiple times for it to 
respond. It's inconsistent and annoying, especially when I'm trying to 
quickly check notifications or start a workout.On the plus side, the 
integration with my smartphone is seamless, and I appreciate being able 
to respond to texts and calls from my wrist. The water resistance has 
also held up well - I've worn it while swimming without any problems.
Customer service has been decent. They were quick to respond when I 
reported the battery issue, but their solution of turning off certain 
features defeats the purpose of having a smartwatch. All in all, while 
there are aspects of the ABC Smartwatch that I really like, the battery 
life and inconsistent performance are major drawbacks. For the price 
point, I expected better. It's not worth your money.
"""

In [23]:
get_review_sentiment(review2)

ENTIRE MODEL OUTPUT: 
<review_breakdown>

a) Key Phrase Extraction:

**Positive Sentiment Phrases:**
- "the design is absolutely gorgeous"
- "It's sleek, lightweight, and looks fantastic on my wrist"
- "The display is crisp and bright, even in direct sunlight"
- "I also love how customizable the watch faces are"
- "The integration with my smartphone is seamless"
- "I appreciate being able to respond to texts and calls from my wrist"
- "The water resistance has also held up well"

**Negative Sentiment Phrases:**
- "The battery life is terrible"
- "I'm lucky if I get through a full day without needing to charge it"
- "This is especially frustrating when I'm traveling or out for long periods"
- "The fitness tracking features, which were a big selling point for me, are hit or miss"
- "It often doesn't accurately count my steps or calculate calories burned during workouts"
- "Another issue I've encountered is with the touch sensitivity"
- "Sometimes it's overly sensitive, registering accide

## Examples

In [24]:
examples = """
Here are some example inputs and outputs to help you understand
the sort of analysis we're looking for
<examples>
<example1>

<customer_review>
I recently bought the ABC Smartwatch, and I have to say, it's been a bit of a rollercoaster. Let's start with the good stuff - the design is absolutely gorgeous. It's sleek, lightweight, and looks fantastic on my wrist. The display is crisp and bright, even in direct sunlight. I also love how customizable the watch faces are, allowing me to switch up the look whenever I want.
However, there are some significant downsides that I can't ignore. The battery life is terrible - I'm lucky if I get through a full day without needing to charge it. This is especially frustrating when I'm traveling or out for long periods. The fitness tracking features, which were a big selling point for me, are hit or miss. It often doesn't accurately count my steps or calculate calories burned during workouts.
Another issue I've encountered is with the touch sensitivity. Sometimes it's overly sensitive, registering accidental touches, while other times I have to tap multiple times for it to respond. It's inconsistent and annoying, especially when I'm trying to quickly check notifications or start a workout.
On the plus side, the integration with my smartphone is seamless, and I appreciate being able to respond to texts and calls from my wrist. The water resistance has also held up well - I've worn it while swimming without any problems.
Customer service has been decent. They were quick to respond when I reported the battery issue, but their solution of turning off certain features defeats the purpose of having a smartwatch.
All in all, while there are aspects of the ABC Smartwatch that I really like, the battery life and inconsistent performance are major drawbacks. For the price point, I expected better. It's not worth your money.
</customer_review>

<ideal_output>
<review_analysis>
a) Key Phrase Extraction:

Positive Phrases:
1. "design is absolutely gorgeous"
2. "sleek, lightweight, and looks fantastic"
3. "display is crisp and bright"
4. "love how customizable the watch faces are"
5. "integration with my smartphone is seamless"
6. "water resistance has held up well"
7. "customer service has been decent"

Negative Phrases:
1. "bit of a rollercoaster"
2. "battery life is terrible"
3. "lucky if I get through a full day"
4. "fitness tracking features are hit or miss"
5. "doesn't accurately count my steps"
6. "touch sensitivity... overly sensitive"
7. "have to tap multiple times"
8. "inconsistent and annoying"
9. "solution of turning off certain features defeats the purpose"
10. "battery life and inconsistent performance are major drawbacks"
11. "not worth your money"

b) Sentiment Analysis:
Arguments for positive sentiment:
- The design and aesthetics of the watch are highly praised
- The display quality is commended
- Customization options are appreciated
- Smartphone integration works well
- Water resistance feature is effective
- Customer service responded quickly

Arguments for negative sentiment:
- Battery life is severely criticized
- Fitness tracking, a key feature, is unreliable
- Touch sensitivity issues cause frustration
- The proposed solution compromises functionality
- The price doesn't justify the performance
- The reviewer explicitly states it's not worth the money

Considering the strength and frequency of sentiments:
- Positive sentiments are mostly about aesthetics and basic functions
- Negative sentiments relate to core functionalities and user experience
- The reviewer uses stronger language for negative aspects ("terrible", "major drawbacks", "not worth your money")
- The conclusion emphasizes disappointment and recommends against purchase

Based on this analysis, the overall sentiment leans negative. While the reviewer appreciates certain aspects, the core issues with battery life, performance, and value for money overshadow the positives. The explicit recommendation against purchase is a strong indicator of overall negative sentiment.

c) Complaint Extraction:
1. Poor battery life
   Quote: "The battery life is terrible - I'm lucky if I get through a full day without needing to charge it"

2. Inaccurate fitness tracking
   Quote: "The fitness tracking features, which were a big selling point for me, are hit or miss. It often doesn't accurately count my steps or calculate calories burned during workouts"

3. Inconsistent touch sensitivity
   Quote: "Sometimes it's overly sensitive, registering accidental touches, while other times I have to tap multiple times for it to respond"

4. Inadequate solution to problems
   Quote: "their solution of turning off certain features defeats the purpose of having a smartwatch"

Total Complaints: 4

d) Summary:
The analysis of the ABC Smartwatch review reveals a predominantly negative sentiment. While the customer appreciates the design, display quality, and some basic features, these positives are outweighed by significant issues with core functionalities. The main complaints center around poor battery life, inaccurate fitness tracking, inconsistent touch sensitivity, and inadequate problem-solving from customer service. The reviewer concludes that the product is not worth the money, emphasizing that the drawbacks are too significant to justify the price point.
</review_analysis>

<json>
{
  "sentiment_score": "Negative",
  "sentiment_analysis": "While the review acknowledges positive aspects like design and basic functionality, the overall sentiment is negative due to significant issues with core features (battery life, fitness tracking, touch sensitivity). The reviewer explicitly states it's 'not worth your money' and emphasizes major drawbacks that overshadow the positive elements.",
  "complaints": [
    "Poor battery life requiring daily charging",
    "Inaccurate fitness tracking features",
    "Inconsistent touch sensitivity causing user interface problems",
    "Inadequate customer service solutions that compromise device functionality"
  ]
}
</json>
</ideal_output>

</example1>
</examples>
"""